## Confirm Implementation of Random/Fixed Effects with Example from Borenstein, 2009.
M. Borenstein, L. V. Hedges, J.P.T. Higgins, H.R. Rothstein, Introduction to Meta-Analysis, John Wiley & Sons, Ltd, Chichester, UK, 2009. https://doi.org/10.1002/9780470743386

#### Imports

In [1]:

from pandas.core.common import SettingWithCopyWarning
from IPython.display import display, Math, Latex
import matplotlib.patches as mpatches                                                       # Figure Legends
import matplotlib.pyplot as plt 
import scipy.stats as stats
import pandas as pd
import numpy as np
import time as t0
import warnings
import copy

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

np.set_printoptions(threshold=np.inf, precision=3, linewidth=300, suppress=True)
np.seterr(divide='ignore', invalid='ignore')
pd.set_option('display.width'      , 1000, 
              'expand_frame_repr'  , False,
              'display.max_rows'   , 999,
              'display.max_columns', 999)

plt.rcParams["font.family" ] = "Myriad Pro"
plt.rcParams["pdf.fonttype"] = 42


#### Meta-Functions

In [2]:
from supp_functions import *

#### Example 1

In [3]:
df_test = pd.DataFrame({'Study'     : ['A'   , 'B'   , 'C'   , 'D'   , 'E'   ],
                        'Hedge\'s G': [ 99.10, 101.20, 101.80,  98.10,  99.10],
                        'Variance'  : [  2.00,   2.00,   0.50,   2.00,   2.00]})

df_test['Final_Weight'] = 1/df_test.Variance

Fixed_Effect     = fixed_effect( df_test, Tcol='Hedge\'s G', Wcol='Final_Weight')
Random_Effect    = random_effect(df_test, Tcol='Hedge\'s G', Wcol='Final_Weight')

print('         Fixed Effect   Random Effect')
print('M    :     {:8.4f}        {:8.4f}'.format(Fixed_Effect['M'      ], Random_Effect['M'      ]))
print('CI-Lw:     {:8.4f}        {:8.4f}'.format(Fixed_Effect['Low'    ], Random_Effect['Low'    ]))
print('CI-Hg:     {:8.4f}        {:8.4f}'.format(Fixed_Effect['Hgh'    ], Random_Effect['Hgh'    ]))
print('Var  :     {:8.4f}        {:8.4f}'.format(Fixed_Effect['Var'    ], Random_Effect['Var'    ]))
print('SE   :     {:8.4f}        {:8.4f}'.format(Fixed_Effect['SE'     ], Random_Effect['SE'     ]))
print('Z    :     {:8.4f}        {:8.4f}'.format(Fixed_Effect['Z'      ], Random_Effect['Z'      ]))
print('Q    :                    {:8.4f}'.format(                         Random_Effect['Q'      ]))
print('p1way:     {:8.4f}        {:8.4f}'.format(Fixed_Effect['P-1Tail'], Random_Effect['P-1Tail']))
print('p2way:     {:8.4f}        {:8.4f}'.format(Fixed_Effect['P-2Tail'], Random_Effect['P-2Tail']))


         Fixed Effect   Random Effect
M    :     100.5875        100.1012
CI-Lw:      99.6075         98.5423
CI-Hg:     101.5675        101.6602
Var  :       0.2500          0.6327
SE   :       0.5000          0.7954
Z    :     201.1750        125.8508
Q    :                      8.4344
p1way:       0.0000          0.0000
p2way:       0.0000          0.0000


#### Example 2

In [9]:
df_test = pd.DataFrame({'Study'     : ['Caroll', 'Grant', 'Peck', 'Donat', 'Stewart', 'Young'],
                        'Hedge\'s G': [.095    , .277   , .367  , .664   , .462     , .185   ],
                        'Variance'  : [.033    , .031   , .050  , .011   , .043     , .023   ]})

df_test['Final_Weight'] = 1/df_test.Variance

Fixed_Effect     = fixed_effect( df_test, Tcol='Hedge\'s G', Wcol='Final_Weight')
Random_Effect    = random_effect(df_test, Tcol='Hedge\'s G', Wcol='Final_Weight')

print('         Fixed Effect   Random Effect')
print('M    :     {:8.4f}        {:8.4f}'.format(Fixed_Effect['M'      ], Random_Effect['M'      ]))
print('CI-Lw:     {:8.4f}        {:8.4f}'.format(Fixed_Effect['Low'    ], Random_Effect['Low'    ]))
print('CI-Hg:     {:8.4f}        {:8.4f}'.format(Fixed_Effect['Hgh'    ], Random_Effect['Hgh'    ]))
print('Var  :     {:8.4f}        {:8.4f}'.format(Fixed_Effect['Var'    ], Random_Effect['Var'    ]))
print('SE   :     {:8.4f}        {:8.4f}'.format(Fixed_Effect['SE'     ], Random_Effect['SE'     ]))
print('Z    :     {:8.4f}        {:8.4f}'.format(Fixed_Effect['Z'      ], Random_Effect['Z'      ]))
print('Q    :                    {:9.4f}'.format(                         Random_Effect['Q'      ]))
print('p1way:     {:8.4f}        {:8.4f}'.format(Fixed_Effect['P-1Tail'], Random_Effect['P-1Tail']))
print('p2way:     {:8.4f}        {:8.4f}'.format(Fixed_Effect['P-2Tail'], Random_Effect['P-2Tail']))


         Fixed Effect   Random Effect
M    :       0.4093          0.3577
CI-Lw:       0.2828          0.1528
CI-Hg:       0.5357          0.5626
Var  :       0.0042          0.0109
SE   :       0.0645          0.1045
Z    :       6.3429          3.4216
Q    :                      11.7433
p1way:       0.0000          0.0003
p2way:       0.0000          0.0006


## Simulation:

#### Compare Against Monte Carlo Style:

In [13]:
def Monte_Carlo(cmb, n=100):
    d_    = np.array([])
    
    for jj in range(cmb.shape[0]):
        a    = np.random.normal(cmb[jj,0], cmb[jj,1], (int(cmb[jj,2] * n)))
        d_   = np.concatenate((d_, a), axis=0)

    u     = np.mean(d_)
    std   = np.std( d_)

    return u, std

In [20]:
def run_sim(print_vals=True, conf_const=False):
    ngrps   = 30
    grdtr   = 9.629
    const   = 10.0

    nmeans  = np.random.uniform(grdtr-1.5, grdtr+1.5, ngrps)
    nstds   = np.random.uniform(0.50, 1.00, ngrps)
    nsubs   = np.random.randint(   4,   50, ngrps)
    
    cmb     = np.zeros([ngrps, 3])

    for ii in range(ngrps):
        s         = np.random.normal(nmeans[ii], nstds[ii], nsubs[ii])
        cmb[ii,0] = np.mean(s)
        cmb[ii,1] = np.std(s)
        cmb[ii,2] = nsubs[ii]
        
    res       = Monte_Carlo(cmb, n=1000)
    u0        = res[0]
    std0      = res[1]
    ci        = 1.96 * std0
    low       = u0 - ci
    hgh       = u0 + ci
    
    df0     = pd.DataFrame({'Mean': nmeans,
                            'Std' : nstds,
                            'N'   : nsubs,
                            'Var' : nstds**2,
                            'Wgt' : 1/(nstds**2)})
    
    nmeans /= const
    nstds  /= const
    df1     = pd.DataFrame({'Mean': nmeans,
                            'Std' : nstds,
                            'N'   : nsubs,
                            'Var' : nstds**2,
                            'Wgt' : 1/(nstds**2)})

    fix0    = fixed_effect( df0, Tcol='Mean', Wcol='Wgt')
    ran0    = random_effect(df0, Tcol='Mean', Wcol='Wgt')
    
    fix1    = fixed_effect( df1, Tcol='Mean', Wcol='Wgt')
    ran1    = random_effect(df1, Tcol='Mean', Wcol='Wgt')
    
    grdtr_  = 9.629 / const
    
    if print_vals == True:
        print('\nDisplay Results')
        print('Idx 0:   Fixed Effect   Random Effect   MonteCarlo   True Value')
        print('Mean :     {:8.4f}       {:8.4f}       {:8.4f}       {:7.4f}'.format(fix0['M'], ran0['M'], u0, grdtr))
        print('Error:     {:8.4f}       {:8.4f}       {:8.4f}'.format(np.abs(fix0['M']-grdtr), np.abs(ran0['M']-grdtr), np.abs(u0-grdtr)))
        print('  ')
        print('CI-Lw:     {:8.4f}       {:8.4f}       {:8.4f}'.format(fix0['Low'   ], ran0['Low'   ], low))
        print('CI-Hg:     {:8.4f}       {:8.4f}       {:8.4f}'.format(fix0['Hgh'   ], ran0['Hgh'   ], hgh))
        print('Var  :     {:8.4f}       {:8.4f}       {:8.4f}'.format(fix0['Var'   ], ran0['Var'   ], std0**2))
        print('SE/SD:     {:8.4f}       {:8.4f}       {:8.4f}'.format(fix0['SE'    ], ran0['SE'    ], std0))
        print('Wgts :     {:8.4f}       {:8.4f}              '.format(fix0['Weight'], ran0['Weight']))

    if conf_const == True:
        fix1  = fixed_effect( df1, Tcol='Mean', Wcol='Wgt')
        ran1  = random_effect(df1, Tcol='Mean', Wcol='Wgt')

        print('\n\nConfirm that results are equivalent if scaled by a constant')

        print('Idx 1:   Fixed Effect   Random Effect   MonteCarlo')
        print('Mean :     {:8.4f}       {:8.4f}     '.format(fix1['M'] * const, ran1['M'] * const, u0 ))
        print('Error:     {:8.4f}       {:8.4f}     '.format(np.abs(fix1['M'] * const-grdtr), np.abs(ran1['M'] * const-grdtr)))
        print('  ')
        print('CI-Lw:     {:8.4f}       {:8.4f}     '.format(fix1['Low'   ] * const, ran1['Low'   ] * const))
        print('CI-Hg:     {:8.4f}       {:8.4f}     '.format(fix1['Hgh'   ] * const, ran1['Hgh'   ] * const))
        print('Var  :     {:8.4f}       {:8.4f}     '.format(fix1['Var'   ] * const, ran1['Var'   ] * const))
        print('SE/SD:     {:8.4f}       {:8.4f}     '.format(fix1['SE'    ] * const, ran1['SE'    ] * const))
        print('Wgts :     {:8.4f}       {:8.4f}     '.format(fix1['Weight'] * const, ran1['Weight'] * const))

run_sim()


Display Results
Idx 0:   Fixed Effect   Random Effect   MonteCarlo   True Value
Mean :       9.5674         9.5633         9.5904        9.6290
Error:       0.0616         0.0657         0.0386
  
CI-Lw:       9.3147         9.2478         7.3852
CI-Hg:       9.8201         9.8787        11.7957
Var  :       0.0166         0.0259         1.2659
SE/SD:       0.1289         0.1609         1.1251
Wgts :      60.1721        38.6168              
